1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
zoo = fetch_ucirepo(id=111) 
  
# data (as pandas dataframes) 
X = zoo.data.features 
y = zoo.data.targets 
  
# metadata 
print(zoo.metadata) 
  
# variable information 
print(zoo.variables)


{'uci_id': 111, 'name': 'Zoo', 'repository_url': 'https://archive.ics.uci.edu/dataset/111/zoo', 'data_url': 'https://archive.ics.uci.edu/static/public/111/data.csv', 'abstract': 'Artificial, 7 classes of animals', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 101, 'num_features': 16, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['type'], 'index_col': ['animal_name'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1990, 'last_updated': 'Fri Sep 15 2023', 'dataset_doi': '10.24432/C5R59V', 'creators': ['Richard Forsyth'], 'intro_paper': None, 'additional_info': {'summary': 'A simple database containing 17 Boolean-valued attributes.  The "type" attribute appears to be the class attribute.  Here is a breakdown of which animals are in which type: (I find it unusual that there are 2 instances of "frog" and one of "girl"!)', 'purpose': None, 'funded_by': None, 'inst

In [6]:
from scipy.special import logsumexp
import numpy as np

class BernoulliMixturewSEM:
    
    def __init__(self, n_components, max_iter, batch_size=10, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.tol = tol
    
    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            for batch in self.iterate_batches():
                self.gamma = self.get_responsibilities(self.get_log_bernoullis(batch))
                self.remember_params()
                self.get_Neff()
                self.get_mu(batch)
                self.get_pi()
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def iterate_batches(self):
        n_samples = len(self.x)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for start_idx in range(0, n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]
            yield self.x.iloc[batch_indices]

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
        self.old_mu = None
        self.old_pi = None
        self.old_gamma = None
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        epsilon = 1e-15
        mu_place = np.clip(mu, epsilon, 1 - epsilon)
        return np.tensordot(x, np.log(mu_place), (1,1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self, batch):
        self.mu = np.einsum('ik,id -> kd', self.gamma, batch) / self.Neff[:, None]
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score
from kmodes.kmodes import KModes

class CustomClusteringEvaluator:
    def __init__(self):
        self.encoded_datasets = {}
    
    def encode_categorical(self, df):
        encoder = LabelEncoder()
        encoded_df = df.copy()
        for column in df.columns:
            if df[column].dtype == 'object':
                encoded_df[column] = encoder.fit_transform(df[column])
        return encoded_df
    
    def evaluate_clustering(self, dataset_name, algorithm, **kwargs):
        dataset = self.encoded_datasets[dataset_name]
        X = dataset.iloc[:, :-1]
        true_labels = dataset.iloc[:, -1]
        
        if algorithm == 'BernoulliMixturewSEM':
            model = BernoulliMixturewSEM(**kwargs)
            model.fit(X)
            labels = model.predict(X)
        elif algorithm == 'KModes':
            km = KModes(**kwargs)
            labels = km.fit_predict(X)
        else:
            raise ValueError("Algorithm not supported.")
        
        fmi, ari, nmi = None, None, None
        
        if true_labels is not None:
            fmi = fowlkes_mallows_score(true_labels, labels)
            ari = adjusted_rand_score(true_labels, labels)
            nmi = normalized_mutual_info_score(true_labels, labels)
        
        return fmi, ari, nmi
    
    def run_evaluation(self, df):
        results = {}
        algorithms = ['BernoulliMixturewSEM', 'KModes']
        dataset_name = 'zoo'
        self.encoded_datasets[dataset_name] = self.encode_categorical(df)
        results[dataset_name] = {}
        for algorithm in algorithms:
            if algorithm == 'BernoulliMixturewSEM':
                kwargs = {'n_components': 2, 'max_iter': 100}
            elif algorithm == 'KModes':
                kwargs = {'n_clusters': 2, 'max_iter': 100}
            fmi, ari, nmi = self.evaluate_clustering(dataset_name, algorithm, **kwargs)
            results[dataset_name][algorithm] = {'FMI': fmi, 'ARI': ari, 'NMI': nmi}
        
        data = []
        for dataset_name, algorithms in results.items():
            if dataset_name == 'zoo':
                for algorithm, metrics in algorithms.items():
                    data.append([dataset_name, algorithm, metrics['FMI'], metrics['ARI'], metrics['NMI']])
        
        results_df = pd.DataFrame(data, columns=['Dataset', 'Algorithm', 'FMI', 'ARI', 'NMI'])
        return results_df

# Instantiate the evaluator and run the evaluation
evaluator = CustomClusteringEvaluator()
results_df = evaluator.run_evaluation(zoo.data.features)
print(results_df)

  Dataset             Algorithm       FMI       ARI       NMI
0     zoo  BernoulliMixturewSEM  0.709483  0.000000  0.000000
1     zoo                KModes  0.672672  0.334538  0.255743


When comparing the performance of the Bernoulli Mixture Model with Stochastic EM Algorithm and the K-Modes Algorithm using FMI, ARI, and NMI, it's clear that FMI consistently gives higher scores than ARI and NMI. This difference is because FMI focuses on cluster similarities through pairwise agreements, leading to higher values when clusters are similar. In contrast, ARI considers both agreements and disagreements, penalizing random assignments; while NMI is sensitive to differences in cluster sizes. Consequently, ARI and NNI may yield lower scores when dealing with imbalanced sizes or partial overlaps between clusters as they capture additional aspects beyond pairwise agreements in clustering.